# Attempt 1: supervised into unsupervised

using random forest to categorise into broad categories and then using unsupervised to further refine into subcategories

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer()
# y = mlb.fit_transform(df_valid_articles['categories'])


In [ ]:
# df_valid_articles = pd.merge(wikipedia_articles, df_valid_articles, on='title', how='inner')

# print(df_valid_articles.head())

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# # fix this

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer(max_features=1000)
# X = tfidf.fit_transform(df_valid_articles['preprocessed_text'])


For multi-label classification, we will try random forest and logistic reg

Random Forest with a OneVsRest strategy

In [ ]:
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.ensemble import RandomForestClassifier

# model = MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)
# model.fit(X_train, y_train)

In [ ]:
# print(model.estimators_)

In [ ]:
# from sklearn.metrics import hamming_loss, accuracy_score

# predictions = model.predict(X_test)
# print(f"Hamming Loss: {hamming_loss(y_test, predictions)}")
# # For exact match, which might be strict for multi-label classification
# print(f"Exact Match Ratio: {accuracy_score(y_test, predictions)}")


The low Hamming Loss indicates that individual label predictions are generally accurate, but the low Exact Match Ratio shows that fully correct multi-label predictions for each sample are rare. This can happen in complex multi-label tasks where getting every label right is difficult due to the intricate relationships between labels.
The discrepancy between the two metrics highlights the challenge of multi-label classification: it's easier to predict some labels correctly than to predict all labels correctly for a given sample.

In [ ]:
# # Load a pre-trained sentence transformer model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # Generate embeddings
# embeddings = model.encode(df_valid_articles['preprocessed_text'].tolist())

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)

# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.ensemble import RandomForestClassifier

# model = MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)
# model.fit(X_train, y_train)


In [ ]:
# from sklearn.metrics import hamming_loss, accuracy_score

# predictions = model.predict(X_test)
# print(f"Hamming Loss: {hamming_loss(y_test, predictions)}")
# # For exact match, which might be strict for multi-label classification
# print(f"Exact Match Ratio: {accuracy_score(y_test, predictions)}")

Revisit Metric Choice: Since EMR is strict (all labels must be correct), consider focusing also on other metrics like F1 score (balance between precision and recall), which might give a more nuanced view of model performance across labels.

# Method 2: Only unsupervised

uses both the categories and the text to cluster into distinct categories

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df_valid_articles['categories'])

In [ ]:
# Perform the inner join
df_valid_articles = pd.merge(qns_answers, df_valid_articles, left_on='article', right_on='title', how='inner')

# Display the first few rows of the merged DataFrame to verify the join
print(df_valid_articles.head())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=1000)
X = tfidf.fit_transform(df_valid_articles['preprocessed_text'])


In [ ]:
from scipy.sparse import hstack

# Assuming X is the TF-IDF matrix and y_encoded is the category matrix
X_combined = hstack([X, y])

In [ ]:
from sklearn.cluster import KMeans

# Decide on the number of clusters
n_clusters = 5  # Example, adjust based on your needs

# Perform clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(X_combined)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# Range of n_clusters values to evaluate
n_clusters_range = range(5, 15)

silhouette_scores = []

for n_clusters in n_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X_combined)
    score = silhouette_score(X_combined, clusters)
    silhouette_scores.append(score)
    print(f"Silhouette Score with n_clusters = {n_clusters}: {score}")

# Plotting the Silhouette Scores
plt.figure(figsize=(10, 6))
plt.plot(n_clusters_range, silhouette_scores, marker='o')
plt.title('Silhouette Scores for Different Numbers of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.show()

# Choose the n_clusters with the highest silhouette score as optimal
optimal_n_clusters_silhouette = n_clusters_range[silhouette_scores.index(max(silhouette_scores))]
print(f"Optimal number of clusters based on silhouette score: {optimal_n_clusters_silhouette}")

There are several ways we can refine this. We can start with dimensionality reduction, before proceeding to word embeddings to capture more semantic content.

In [ ]:
from sklearn.decomposition import TruncatedSVD
# Dimensionality reduction for efficiency and to enhance clustering performance
svd = TruncatedSVD(n_components=100, random_state=42)
X_reduced = svd.fit_transform(X_combined)

In [ ]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(df_valid_articles['preprocessed_text'].tolist())

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Define a function to compute KMeans clustering and silhouette scores
def compute_silhouette_scores(X, n_clusters_range):
    silhouette_scores = []
    for n_clusters in n_clusters_range:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        clusters = kmeans.fit_predict(X)
        score = silhouette_score(X, clusters)
        silhouette_scores.append(score)
    return silhouette_scores

# Define the range of n_clusters
n_clusters_range = np.arange(10, 101, 5)

# Assuming X_combined, X_reduced, and embeddings are defined
# Compute silhouette scores for each model
silhouette_scores_combined = compute_silhouette_scores(X_combined, n_clusters_range)
silhouette_scores_reduced = compute_silhouette_scores(X_reduced, n_clusters_range)
silhouette_scores_embeddings = compute_silhouette_scores(embeddings, n_clusters_range)

# Plotting the Silhouette Scores
plt.figure(figsize=(12, 8))

plt.plot(n_clusters_range, silhouette_scores_combined, marker='o', label='X_combined')
plt.plot(n_clusters_range, silhouette_scores_reduced, marker='x', label='X_reduced')
plt.plot(n_clusters_range, silhouette_scores_embeddings, marker='^', label='Embeddings')

plt.title('Silhouette Scores for Different Models')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.legend()
plt.grid(True)
plt.show()

# Determine the optimal number of clusters for each model
optimal_n_clusters_combined = n_clusters_range[np.argmax(silhouette_scores_combined)]
optimal_n_clusters_reduced = n_clusters_range[np.argmax(silhouette_scores_reduced)]
optimal_n_clusters_embeddings = n_clusters_range[np.argmax(silhouette_scores_embeddings)]

print(f"Optimal number of clusters for X_combined: {optimal_n_clusters_combined}")
print(f"Optimal number of clusters for X_reduced: {optimal_n_clusters_reduced}")
print(f"Optimal number of clusters for embeddings: {optimal_n_clusters_embeddings}")


In [ ]:
# Clustering with the optimal number of clusters
kmeans_optimal = KMeans(n_clusters=optimal_n_clusters_reduced, random_state=42)
df_valid_articles['cluster'] = kmeans_optimal.fit_predict(X_reduced)

# Qualitative Analysis of the Clustered Articles
for i in range(optimal_n_clusters_reduced):
    print(f"Cluster {i} titles:")
    print(df_valid_articles[df_valid_articles['cluster'] == i]['title'].head(10), "\n")  # Displaying the first 10 titles per cluster



# Keyword Matching Models

data preprocessing

In [ ]:
def extract_keywords(text):
    tokens = simple_preprocess(text, deacc=True) # remove punctuation, see below for notes
    stopwords_list = set(stopwords.words('english'))
    keywords = [word for word in tokens if word not in stopwords_list]
    return keywords

In [ ]:
train_qns['keywords'] = train_qns['question'].apply(extract_keywords)
train_qns[['question', 'keywords']].head()

In [ ]:
articles['keywords'] = articles['text'].apply(extract_keywords)
articles[['title', 'keywords']].head()

let's create a merged dataset for easier visualisation

In [ ]:
merged_data = train_qns.merge(articles, left_on='article', right_on='title')
merged_data['question_keywords'] = merged_data['keywords_x']
merged_data['article_keywords']= merged_data['keywords_y']
merged_data['question_keywords_str'] = merged_data['question_keywords'].apply(lambda x: ' '.join(x))
merged_data['article_keywords_str'] = merged_data['article_keywords'].apply(lambda x: ' '.join(x))
merged_data = merged_data.drop(['keywords_x', 'keywords_y'], axis=1)
merged_data.head()

In [ ]:
lemmatizer = WordNetLemmatizer()

def normalize_and_lemmatize(keyword_list):
    normalized = [re.sub(r'[^a-zA-Z]', '', word.lower()) for word in keyword_list]
    lemmatized = [lemmatizer.lemmatize(word) for word in normalized if word]
    return lemmatized

keyword_matching_dataset = merged_data.copy()
keyword_matching_dataset['question_keywords'] = keyword_matching_dataset['question_keywords'].apply(normalize_and_lemmatize)
keyword_matching_dataset['article_keywords'] = keyword_matching_dataset['article_keywords'].apply(normalize_and_lemmatize)

keyword_matching_dataset['question_keywords_str'] = keyword_matching_dataset['question_keywords'].apply(lambda x: ' '.join(x))
keyword_matching_dataset['article_keywords_str'] = keyword_matching_dataset['article_keywords'].apply(lambda x: ' '.join(x))

In [ ]:
keyword_matching_dataset.head()

TI-IDF

use of cosine similarity

Measure of Orientation, Not Magnitude: Cosine similarity focuses on the angle between two vectors, not their magnitude. This means it can effectively determine how similar two documents are in terms of their content, regardless of their length.

Robust to Different Lengths: Since it's based on vector orientation, cosine similarity is less affected by the length of documents. This is particularly useful when comparing texts of different lengths.

Intuitive Interpretation: The cosine similarity score ranges from -1 to 1, where 1 indicates identical directionality (and thus, high similarity), 0 indicates no similarity, and -1 indicates completely opposite. This makes the results relatively straightforward to interpret.

Effectiveness in High-Dimensional Space: Text data, especially when vectorized using methods like TF-IDF, often exists in a high-dimensional space. Cosine similarity is effective in such spaces, where Euclidean distance can become less meaningful due to the curse of dimensionality.

In [ ]:
# initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_questions = tfidf_vectorizer.fit_transform(keyword_matching_dataset['question_keywords_str'])
tfidf_matrix_articles = tfidf_vectorizer.transform(keyword_matching_dataset['article_keywords_str'])

# calculate cosine similarity between each question and its corresponding article
cosine_sim_tfidf = [cosine_similarity(tfidf_matrix_questions[i], tfidf_matrix_articles[i])[0][0] for i in range(len(keyword_matching_dataset))]

In [ ]:
# initialize TF-IDF Vectorizer with n-grams (e.g., unigrams and bigrams)
tfidf_vectorizer_ngrams = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix_questions_ngrams = tfidf_vectorizer_ngrams.fit_transform(keyword_matching_dataset['question_keywords_str'])
tfidf_matrix_articles_ngrams = tfidf_vectorizer_ngrams.transform(keyword_matching_dataset['article_keywords_str'])

# calculate cosine similarity between each question and its corresponding article
cosine_sim_ngrams = [cosine_similarity(tfidf_matrix_questions_ngrams[i], tfidf_matrix_articles_ngrams[i])[0][0] for i in range(len(keyword_matching_dataset))]

In [ ]:
def calculate_bm25_for_pair(query, article_text, bm25_model):
    query_tokens = query.split(" ")
    article_tokens = article_text.split(" ")
    doc_scores = bm25_model.get_scores(query_tokens)
    article_index = tokenized_corpus.index(article_tokens)
    return doc_scores[article_index]

tokenized_corpus = [doc.split(" ") for doc in merged_data['article_keywords_str']]
bm25_model = BM25Okapi(tokenized_corpus)

In [ ]:
@delayed # for lazy execution, sets up task graph
def delayed_calculate_bm25_for_pair(query, article_text, bm25_model): # delayed task
    return calculate_bm25_for_pair(query, article_text, bm25_model)

dask_df = dd.from_pandas(merged_data, npartitions=4) # adjust partitions 

bm25_scores = [delayed_calculate_bm25_for_pair(row['question_keywords_str'], row['article_keywords_str'], bm25_model) for index, row in dask_df.iterrows()] # series of independent task for each row

bm25_results = dd.compute(*bm25_scores) # executes the parallel processing

In [ ]:
keyword_matching_dataset['cosine_sim_tfidf'] = cosine_sim_tfidf
keyword_matching_dataset['cosine_sim_ngrams'] = cosine_sim_ngrams
keyword_matching_dataset['bm25_score'] = bm25_results

keyword_matching_dataset.head()

In [ ]:
bm25_score_reshaped = np.array(keyword_matching_dataset['bm25_score']).reshape(-1, 1)
scaler = MinMaxScaler()
keyword_matching_dataset['norm_bm25_score'] = scaler.fit_transform(bm25_score_reshaped)

keyword_matching_dataset.head()

In [ ]:
mean_sim_tfidf = np.mean(keyword_matching_dataset['cosine_sim_tfidf'])
mean_sim_ngrams = np.mean(keyword_matching_dataset['cosine_sim_ngrams'])
mean_norm_bm25_score = np.mean(keyword_matching_dataset['norm_bm25_score'])

print(f"Mean Cosine Similarity (TF-IDF): {mean_sim_tfidf}")
print(f"Mean Cosine Similarity (N-grams): {mean_sim_ngrams}")
print(f"Mean BM25 Score: {mean_norm_bm25_score}")

seems like both are performing poorly. we can look at this distribution further

In [ ]:
plt.scatter(keyword_matching_dataset['points'], keyword_matching_dataset['cosine_sim_tfidf'], label='TF-IDF', alpha=0.5)
plt.scatter(keyword_matching_dataset['points'], keyword_matching_dataset['cosine_sim_ngrams'], label='N-grams', alpha=0.5)
plt.scatter(keyword_matching_dataset['points'], keyword_matching_dataset['norm_bm25_score'], label='BM25', alpha=0.5)

plt.xlabel('Points')
plt.ylabel('Cosine Similarity')
plt.title('Correlation between Points and Cosine Similarity')
plt.legend()
plt.show()


no clear pattern, points scatter across almost all possible values. ti-idf outperforms n-grams by a marginal extent and bm25 to a greater extent

In [ ]:
high_similarity_threshold = 0.8
low_similarity_threshold = 0.2

In [ ]:
# High-points, high-similarity outliers for TF-IDF
high_points_high_tfidf = keyword_matching_dataset[
    (keyword_matching_dataset['points'] > keyword_matching_dataset['points'].quantile(0.9)) & 
    (keyword_matching_dataset['cosine_sim_tfidf'] > high_similarity_threshold)
]

# High-points, low-similarity outliers for TF-IDF
high_points_low_tfidf = keyword_matching_dataset[
    (keyword_matching_dataset['points'] > keyword_matching_dataset['points'].quantile(0.9)) & 
    (keyword_matching_dataset['cosine_sim_tfidf'] < low_similarity_threshold)
]

# High-points, high-similarity outliers for N-grams
high_points_high_ngrams = keyword_matching_dataset[
    (keyword_matching_dataset['points'] > keyword_matching_dataset['points'].quantile(0.9)) & 
    (keyword_matching_dataset['cosine_sim_ngrams'] > high_similarity_threshold)
]

# High-points, low-similarity outliers for N-grams
high_points_low_ngrams = keyword_matching_dataset[
    (keyword_matching_dataset['points'] > keyword_matching_dataset['points'].quantile(0.9)) & 
    (keyword_matching_dataset['cosine_sim_ngrams'] < low_similarity_threshold)
]

# High-points, high-similarity outliers for BM25
high_points_high_bm25 = keyword_matching_dataset[
    (keyword_matching_dataset['points'] > keyword_matching_dataset['points'].quantile(0.9)) & 
    (keyword_matching_dataset['bm25_score'] > high_similarity_threshold)
]

# High-points, low-similarity outliers for BM25

high_points_low_bm25 = keyword_matching_dataset[
    (keyword_matching_dataset['points'] > keyword_matching_dataset['points'].quantile(0.9)) & 
    (keyword_matching_dataset['bm25_score'] < low_similarity_threshold)
]

In [ ]:
high_points_high_tfidf.head()

In [ ]:
high_points_low_tfidf.head()

as suspected, some questions with very high points have answers in a very specific part of an article which would not have the same keywords as the question (**rephrase**)

In [ ]:
high_points_high_ngrams.head()

In [ ]:
high_points_low_ngrams.head()

exactly the same performance

In [ ]:
high_points_high_bm25.head()

look at results

In [ ]:
high_points_low_bm25.head()

same performance as other two models

we should move onto models that capture the semantic content behind the words

embedding models

In [ ]:
semantic_matching_dataset = merged_data.copy()

semantic_matching_dataset.head()

In [ ]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'tagger', 'ner'])

def get_spacy_vectors(texts, batch_size=100):
    vectors = []
    for doc in nlp.pipe(texts, batch_size=batch_size):
        vectors.append(doc.vector)
    return np.array(vectors)

questions = semantic_matching_dataset['question'].tolist()
articles = semantic_matching_dataset['text'].tolist()

question_vectors_spacy = get_spacy_vectors(questions)
article_vectors_spacy = get_spacy_vectors(articles)
cosine_similarities_spacy = cosine_similarity(question_vectors_spacy, article_vectors_spacy)
cosine_sim_scores = np.diag(cosine_similarities_spacy)
semantic_matching_dataset['cosine_sim_spacy'] = cosine_sim_scores

In [ ]:
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def encode_texts_in_batches(texts, model, batch_size=128):
    vectors = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        # ensure the model's output is converted to tensors and moved to CPU before converting to numpy
        batch_vectors = model.encode(batch, convert_to_tensor=True).cpu().numpy()
        vectors.extend(batch_vectors)
    return np.array(vectors)

question_vectors_sbert = encode_texts_in_batches(
    semantic_matching_dataset['question'].tolist(), sbert_model)
article_vectors_sbert = encode_texts_in_batches(
    semantic_matching_dataset['text'].tolist(), sbert_model)

cosine_sim_scores = [cosine_similarity([q_vec], [a_vec])[0][0] for q_vec, a_vec in zip(question_vectors_sbert, article_vectors_sbert)]
semantic_matching_dataset['cosine_sim_sbert'] = cosine_sim_scores


In [ ]:
# may need to delete cached models before running use model
# !rm -rf /var/folders/j3/2xy_ffxd6jq618phh49pk1b00000gn/T/tfhub_modules/*

use_model = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')


def batch_get_use_vector(texts, batch_size=32):
    vectors = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_vectors = use_model(batch_texts).numpy()
        vectors.append(batch_vectors)
    return np.vstack(vectors)

questions = semantic_matching_dataset['question'].tolist()
articles = semantic_matching_dataset['text'].tolist()

semantic_matching_dataset['question_vector_use'] = list(map(list, batch_get_use_vector(questions)))
semantic_matching_dataset['article_vector_use'] = list(map(list, batch_get_use_vector(articles)))

cosine_sims = np.array([
    cosine_similarity([q_vec], [a_vec])[0][0] 
    for q_vec, a_vec in zip(semantic_matching_dataset['question_vector_use'], semantic_matching_dataset['article_vector_use'])
])

semantic_matching_dataset['cosine_sim_use'] = cosine_sims

In [ ]:
semantic_matching_dataset.head()

In [ ]:
mean_sim_spacy = np.mean(semantic_matching_dataset['cosine_sim_spacy'])
mean_sim_sbert = np.mean(semantic_matching_dataset['cosine_sim_sbert'])
mean_sim_use = np.mean(semantic_matching_dataset['cosine_sim_use'])

print(f"Mean Cosine Similarity (SpaCy): {mean_sim_spacy}")
print(f"Mean Cosine Similarity (SBERT): {mean_sim_sbert}")
print(f"Mean Cosine Similarity (USE): {mean_sim_use}")

spacy embedding model clearly outperforms

In [ ]:
plt.scatter(semantic_matching_dataset['points'], semantic_matching_dataset['cosine_sim_spacy'], label='SpaCy', alpha=0.5)
plt.scatter(semantic_matching_dataset['points'], semantic_matching_dataset['cosine_sim_sbert'], label='SBERT', alpha=0.5)
plt.scatter(semantic_matching_dataset['points'], semantic_matching_dataset['cosine_sim_use'], label='USE', alpha=0.5)

plt.xlabel('Points')
plt.ylabel('Cosine Similarity')
plt.title('Correlation between Points and Cosine Similarity')
plt.legend()
plt.show()

seems that spacy model captures a good portion of the lower scored questions. let's explore the question-article pairs that scored a near perfect 1.00 similarity score, as well as the opposite

In [ ]:
semantic_matching_dataset[semantic_matching_dataset['cosine_sim_spacy'] >= 0.9]


Direct Question-Answer Relation: Each question directly relates to the topic of the article. The keywords from the questions are explicitly present in the text of the articles, leading to a higher similarity score.

Specificity: The questions are specific, and the articles contain detailed information that directly answers these questions. This specificity likely results in a higher concentration of relevant terms and less noise, which boosts similarity scores.

Relevance of Content: The content of the questions closely aligns with the main subject of the articles. For example, a question about the environment directly relates to an article titled "Environment," and a question about MRI accuracy directly relates to an article about "Magnetic resonance imaging."

High readability scores as well

In [ ]:
semantic_matching_dataset[semantic_matching_dataset['cosine_sim_spacy'] <= 0.0]

Mismatch of Context: The questions are about specific events or people, while the articles titled with years likely cover a wide range of events that happened in that year. The spaCy embeddings might not find a strong contextual match between the question's focus and the broad content of the article.

Lack of Specificity in Articles: Articles that are simply titled with a year might not have a strong thematic focus, leading to a diluted set of vectors that don't match well with the more focused vectors generated from the questions.

Semantic Ambiguity: Years by themselves do not carry specific semantic information. Without additional context, the model may struggle to link a question about a specific event or individual to an article that broadly covers everything related to that year.

Named Entity Recognition (NER) Challenges: If the model is heavily weighing named entities, such as specific names and dates, it may not correctly associate the relevance of the year to the specific event or individual in the question.

Temporal Relevance: Questions that ask about a particular date or event may not align well with the content of an article that summarizes an entire year. The temporal focus is too broad in the article to match the specificity of the question.

given that we are not able to modify the training and test data set, fine tuning the data for the spacy model would hence be quite challenging. we could instead look for alternative models that are specifically designed for chatbots (most common use case of question-answer)

In [ ]:
def print_memory_usage():
    available_memory = psutil.virtual_memory().available
    total_memory = psutil.virtual_memory().total
    used_memory = total_memory - available_memory
    print(f"Used memory: {used_memory / (1024**3):.2f} GB")
    print(f"Available memory: {available_memory / (1024**3):.2f} GB")
    print(f"Total memory: {total_memory / (1024**3):.2f} GB")

print_memory_usage()

In [ ]:
del nlp, use_model, sbert_model
del question_vectors_spacy, article_vectors_spacy
del question_vectors_sbert, article_vectors_sbert
del tfidf_matrix_questions, tfidf_matrix_articles, cosine_sims, cosine_sim_scores

columns_to_drop = ['question_vector_use', 'article_vector_use', 'question_vector_spacy', 'article_vector_spacy', 'question_vectors_sbert', 'article_vectors_sbert', 'cosine_sim_use', 'cosine_sim_spacy', 'cosine_sim_sbert']
semantic_matching_dataset = semantic_matching_dataset.drop(columns=columns_to_drop, errors='ignore')

gc.collect()
print_memory_usage()

In [ ]:
transformer_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

def encode_documents(documents):
    print('Encoding documents...')
    encoded_input = tokenizer(documents, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return embeddings.cpu().detach().numpy()

def mean_pooling(model_output, attention_mask):
    print('Mean pooling...')
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

corpus = semantic_matching_dataset['text'].values.tolist()
corpus_embeddings = encode_documents(corpus)

index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)


def search(query, k=5):
    query_embedding = transformer_model.encode(query)
    distances, indices = index.search(np.array([query_embedding]), k)
    return [(corpus[i], distances[0][j]) for j, i in enumerate(indices[0])]

query = semantic_matching_dataset['question']
relevant_docs = search(query)
print(relevant_docs)